<a href="https://colab.research.google.com/github/mariarua/Fake-job-posting/blob/main/Fake_job_posting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
import nltk
from nltk.corpus import stopwords

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!git clone https://github.com/mariarua/Fake-job-posting /content/drive/MyDrive/fake_job_posting

Cloning into '/content/drive/MyDrive/fake_job_posting'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 45 (delta 17), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (45/45), 16.01 MiB | 5.07 MiB/s, done.


In [21]:
Jobs_Dataset = pd.read_csv('/content/drive/MyDrive/fake_job_posting/fake_job_postings.csv')
jobs_DS = Jobs_Dataset.copy()

**Exploración de datos**

Comenzaremos buscando aquellas características que tenga en 60% de los datos en null y las quitares de nuestro dataset

In [24]:
jobs_DS.isnull().mean()<0.4

job_id                  True
title                   True
location                True
department             False
salary_range           False
company_profile         True
description             True
requirements            True
benefits               False
telecommuting           True
has_company_logo        True
has_questions           True
employment_type         True
required_experience     True
required_education     False
industry                True
function                True
fraudulent              True
dtype: bool

In [ ]:
jobs_DS= jobs_DS[jobs_DS.columns[jobs_DS.isnull().mean()<0.4]]

Después de esto rellenaremos el null por un string vacío

In [ ]:
jobs_DS.fillna('',inplace=True)

Eliminaremos algunas columnas de nuestro dataset debido a que no son muy útiles para lo que deseamos realizar

In [ ]:
jobs_DS.drop(columns=['telecommuting','has_company_logo','has_questions','job_id'], inplace=True)

Para procesar el texto vamos a colocar todos las características juntas en una columna que llamaremos text. Y eliminaremos el resto de las columnas, de este modo tendremos todas las características en un mismo lugar

In [ ]:
jobs_DS['text'] = jobs_DS['title']+" "+jobs_DS['location']+" "+jobs_DS['company_profile']+" "+jobs_DS['description']+" "+jobs_DS['requirements']+" "+jobs_DS['employment_type']+" "+jobs_DS['required_experience']+" "+jobs_DS['industry']+" "+jobs_DS['function']

In [ ]:
jobs_DS.drop(columns=['title','location','company_profile','description','requirements','employment_type','required_experience','industry','function'],inplace=True)

Luego vamos a reemplazar los saltos de línea, salto de linea y retorno, y los espacios que son tabs

In [ ]:
jobs_DS['text'] = jobs_DS['text'].str.replace('\n', ' ')
jobs_DS['text'] = jobs_DS['text'].str.replace('\r', ' ')
jobs_DS['text'] = jobs_DS['text'].str.replace('\t', ' ')

Ahora, vamos a remover los números y los caracteres especiales

In [ ]:
jobs_DS['text'] = jobs_DS['text'].apply(lambda x: re.sub(r'[0-9]',' ',x))
jobs_DS['text'] = jobs_DS['text'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))

Y colocar todo el texto en minúsculas 

In [ ]:
jobs_DS['text']= jobs_DS['text'].apply(lambda s:s.lower() if type(s) == str else s)